In [1]:

import torch
from torch import nn

from tqdm import tqdm as tqdm
import matplotlib.pyplot as plt
import sys
sys.path.append('../src')
sys.path.append('../')

import importlib
import sys
from torch.utils.data import  Dataset
from sklearn.metrics import roc_auc_score
import torch.nn.functional as F
import pytorch_lightning as pl

import pandas as pd
import seaborn as sn
import torch
import numpy as np
from OpenOODmain.openood.evaluators.metrics  import compute_all_metrics
from scipy.stats import multivariate_normal
 
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans 


In [29]:
blue ="#004488"#  "#1f77b4"
green = "#009E73" #2ca02c"
red = "#DDAA33"#"#ff7f0e"
tr = "train"
te = "test"
ood = "OOD"

In [30]:
import csv
import os
# functions _save_csv and save_table are adapted from https://github.com/Jingkang50/OpenOOD/blob/main/openood/evaluators/ood_evaluator.py
def _save_csv(metrics, dataset_name, output_folder,t,method):
        [fpr, auroc, aupr_in, aupr_out, accuracy] = metrics

        write_content = {
            'method' : method,
            'dataset': dataset_name,
            'FPR@95': '{:.2f}'.format(100 * fpr),
            'AUROC': '{:.2f}'.format(100 * auroc),
            'AUPR_IN': '{:.2f}'.format(100 * aupr_in),
            'AUPR_OUT': '{:.2f}'.format(100 * aupr_out),
            'ACC': '{:.2f}'.format(100 * accuracy),
            "timestep" : '{:.2f}'.format(100 * t)
        }

        fieldnames = list(write_content.keys())

        # print ood metric results
        print('FPR@95: {:.2f}, AUROC: {:.2f}'.format(100 * fpr, 100 * auroc),
              end=' ',
              flush=True)
        print('AUPR_IN: {:.2f}, AUPR_OUT: {:.2f}'.format(
            100 * aupr_in, 100 * aupr_out),
              flush=True)
        print('ACC: {:.2f}'.format(accuracy * 100), flush=True)
        print(u'\u2500' * 70, flush=True)

        csv_path = os.path.join(output_folder, 'ood.csv')
        if not os.path.exists(csv_path):
            with open(csv_path, 'w', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerow(write_content)
        else:
            with open(csv_path, 'a', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writerow(write_content)

def save_table(path):
    df = pd.read_csv(f'{path}/ood.csv')
    # Filter out rows where ACC is 0 or 100
    df_filtered = df[(df['ACC'] != 0) & (df['ACC'] != 100)]
    # For each dataset, pick the row with the highest AUROC
    best_rows = df_filtered.loc[df_filtered.groupby('dataset')['AUROC'].idxmax()]
    # Reset index for clarity
    best_rows.reset_index(drop=True, inplace=True)
    # Display the resulting table
    print(best_rows)
    # Optionally save the result to a new CSV
    best_rows.to_csv('highest_auroc_per_dataset.csv', index=False)

def softmax_smaller_better(scores, alpha=1.0, smaller_better = True):
    scores = np.array(scores, dtype=float)
    axis = 0 if scores.ndim == 1 else -1

    # 1) flip sign so smaller scores → larger exponents
    z = -alpha * scores if smaller_better else alpha * scores

    # 2) subtract the max over axis for numerical stability
    z_max = np.max(z, axis=axis, keepdims=True)
    z = z - z_max

    # 3) exponentiate and normalize
    exps = np.exp(z)
    denom = np.sum(exps, axis=axis, keepdims=True)

    # 4) safe‐divide (if denom is ever zero, replace by 1 to avoid div/0)
    denom = np.where(denom == 0, 1.0, denom)
    probs = exps / denom

    return probs


def btb(scores):
    return  scores.max(axis = 0 if scores.ndim == 1 else -1, keepdims=True) - scores + scores.min(axis = 0 if scores.ndim == 1 else -1, keepdims=True)
    #return (scores - scores.min(axis = 0 if scores.ndim == 1 else -1, keepdims=True))/(scores.max(axis = 0 if scores.ndim == 1 else -1, keepdims=True)- scores.min(axis = 0 if scores.ndim == 1 else -1, keepdims=True))

def get_time_t(tag,top_k,num_step,reps,t_ind,output_folder):
    #e.g. (30,128)
    j = 0
    dif = np.load(f'{output_folder}/dif_loss_{tag}_steps_{num_step}_reps_{reps}_{j}.npy')
    logp = np.load(f'{output_folder}/log_probs_{tag}_steps_{num_step}_reps_{reps}_{j}.npy')
    rec = np.load(f'{output_folder}/rec_loss_{tag}_steps_{num_step}_reps_{reps}_{j}.npy')
    dif_t = [dif[t_ind]]
    logp_t = [logp[t_ind]]
    rec_t = [rec[t_ind]]


    for j in range(top_k-1):
        j= j + 1
        dif_t.append(np.load(f'{output_folder}/dif_loss_{tag}_steps_{num_step}_reps_{reps}_{j}.npy')[t_ind])
        logp_t.append(np.load(f'{output_folder}/log_probs_{tag}_steps_{num_step}_reps_{reps}_{j}.npy')[t_ind])
        rec_t.append(np.load(f'{output_folder}/rec_loss_{tag}_steps_{num_step}_reps_{reps}_{j}.npy')[t_ind])            
   

    return np.array(dif_t), np.array(logp_t), np.array(rec_t)


In [53]:
#a function that draw diffusion losses and TKDL scores next to each other
def draw_basics_cw(tag="", ldm_version=10, epoch=2999, reps=25, num_step=10,
                acts_input=None,
                gen_softs=None,
                ind_name=None,
                output_folder=None,top_k=10,t_lim=0.,input_folder = None):
    ood_softs = np.load(f"{acts_input}/resnet18_{ind_name}_{tag}_softs.npy")
    test_softs = np.load(f"{acts_input}/resnet18_{ind_name}_test_softs.npy")

    ood_y = np.load(f"{acts_input}/resnet18_{ind_name}_{tag}_y.npy")
    test_y = np.load(f"{acts_input}/resnet18_{ind_name}_test_y.npy")
    # Load data
    labels = np.concatenate([test_y, ood_y])
    pred = np.concatenate([test_softs.argmax(1), ood_softs.argmax(1)])    
    for t_ind in range(num_step):
        t_ind_nr = 1. - t_ind*(1-t_lim)/num_step

        if t_ind_nr < 0.9:
            continue
    #    dif_gen,logp_gen,rec_gen = get_time_t("gen",top_k,num_step,reps,t_ind,output_folder)
        dif_test,logp_test,rec_test = get_time_t("test",top_k,num_step,reps,t_ind,input_folder)
        dif_train,logp_train,rec_train = get_time_t("train",top_k,num_step,reps,t_ind,input_folder)
        dif_ood,logp_ood,rec_ood = get_time_t(tag,top_k,num_step,reps,t_ind,input_folder) #shape (top_k,bs)
        plt.rcParams.update({
            'font.size': 12,          # base font size
            'axes.titlesize': 18,     # title font size
            'axes.labelsize': 18,     # x– and y–label font size
            'xtick.labelsize': 14,    # tick label font size
            'ytick.labelsize': 14,    # tick label font size
            'legend.fontsize': 16,    # legend font size
            'figure.titlesize': 10    # figure title
        })
    # Create a figure with 6 subplots arranged in 2 rows x 3 columns
        fig, axs = plt.subplots(1, 2, figsize=(12, 5),constrained_layout=True)

        axs = axs.flatten()  # flatten to simplify indexing

    # ----------------- Plot 1: Dif Loss  -----------------
        ax = axs[0]
#        ax.hist(dif_gen.min(0), bins=100, alpha=0.5, density=True, label="gen")
        ax.hist(dif_train.min(0), bins=100, alpha=0.5, density=True, label=tr,color = blue,hatch=r'\\')
        ax.hist(dif_test.min(0), bins=100, alpha=0.5, density=True, label=te,color = green,hatch=r'//')
        ax.hist(dif_ood.min(0), bins=100, alpha=0.5, density=True, label=ood, color = red,hatch=r'||')
        total_distances = np.concatenate([dif_test.min(0), dif_ood.min(0)])
        results = compute_all_metrics(total_distances, labels, pred)
        auc =results[1].round(3)
        if results[1].round(3) < 0.5:
            auc = (1- auc).round(3)
        method = f"L_diff_{t_ind_nr}"
        _save_csv(results, tag, output_folder, t_ind_nr,method)
        ax.set_title("AUROC = " + str(auc) )
        ax.set_xlabel("Dif. Loss")   # ← X‐axis label
        ax.set_ylabel("Density")  
        ax.legend()
     # ----------------- Plot 2: TKDL -----------------      
        sm_test= softmax_smaller_better(dif_test.T).max(1) 
        sm_train = softmax_smaller_better(dif_train.T).max(1) 
        sm_ood = softmax_smaller_better(dif_ood.T).max(1) 
        
        ax = axs[1]
   #     ax.hist(sm_gen, bins=100, alpha=0.5, density=True, label="gen")
        ax.hist(sm_train, bins=100, alpha=0.5, density=True, label=tr,color = blue,hatch=r'\\')
        ax.hist(sm_test, bins=100, alpha=0.5, density=True, label=te,color = green,hatch=r'//')
        ax.hist(sm_ood, bins=100, alpha=0.5, density=True, label=ood,color = red,hatch=r'||')
        total_distances = np.concatenate([sm_test, sm_ood])
        results = compute_all_metrics(total_distances, labels, pred)
        auc =results[1].round(3)
        if results[1].round(3) < 0.5:
            auc = (1- auc).round(3)
        method = f"L_diffSM_{t_ind_nr}"
        _save_csv(results, tag, output_folder, t_ind_nr,method)
        ax.set_title("AUROC = " + str(auc) )
        ax.set_xlabel("TKDL")   # ← X‐axis label
        ax.set_ylabel("Density")
        ax.legend()
 
        if ind_name == "cif10":
            title_name = "CIFAR-10"
        elif ind_name == "cif100":
            title_name = "CIFAR-100"
        else:
            title_name = "ImageNet200"

        #plt.suptitle(f"VDM trained = {title_name}; OOD = {tag}; Diffusion Loss; epoch = {epoch};  time = {t_ind_nr}")
        plt.savefig(f"{output_folder}/LDM_ood_rep_{reps}_{ind_name}_{tag}_time_{t_ind_nr}_{epoch}.png", dpi=300)
        plt.clf()
        plt.close()



In [52]:
# a function that draws PL and EL likelihoods next to each other
def draw_densities(tag="", ldm_version=10, epoch=2999, acts_input=None,
                gen_softs=None,
                ind_name=None,
                output_folder=None,input_folder = None):
    ood_softs = np.load(f"{acts_input}/resnet18_{ind_name}_{tag}_softs.npy")
    #train_softs = np.load(f"{acts_input}/resnet18_{ind_name}_train_softs.npy")
    test_softs = np.load(f"{acts_input}/resnet18_{ind_name}_test_softs.npy")

    ood_y = np.load(f"{acts_input}/resnet18_{ind_name}_{tag}_y.npy")
    test_y = np.load(f"{acts_input}/resnet18_{ind_name}_test_y.npy")
    
    #densities_gen = np.load(f'{output_folder}/densities_gen.npy')
    #uncond_densities_gen = np.load(f'{output_folder}/uncond_densities_gen.npy')
    
    densities_test = np.load(f'{input_folder}/densities_test.npy')
    uncond_densities_test = np.load(f'{input_folder}/uncond_densities_test.npy')

    densities_train = np.load(f'{input_folder}/densities_train.npy')
    uncond_densities_train = np.load(f'{input_folder}/uncond_densities_train.npy')
    
    densities_ood = np.load(f'{input_folder}/densities_{tag}.npy')
    uncond_densities_ood = np.load(f'{input_folder}/uncond_densities_{tag}.npy')


    labels = np.concatenate([test_y, ood_y])
    pred = np.concatenate([test_softs.argmax(1), ood_softs.argmax(1)])
    plt.rcParams.update({
            'font.size': 12,          # base font size
            'axes.titlesize': 20,     # title font size
            'axes.labelsize': 18,     # x– and y–label font size
            'xtick.labelsize': 14,    # tick label font size
            'ytick.labelsize': 14,    # tick label font size
            'legend.fontsize': 16,    # legend font size
            'figure.titlesize': 10    # figure title
        })   
    # Create a figure with 6 subplots arranged in 2 rows x 3 columns
    fig, axs = plt.subplots(1, 2, figsize=(12, 5),constrained_layout=True)
    axs = axs.flatten()  # flatten to simplify indexing

    # ----------------- Plot 4: logp0 of UNcond density-----------------
    ax = axs[0]
    #ax.hist(uncond_densities_gen[0], bins=100, alpha=0.5, density=True, label="gen")
    ax.hist(uncond_densities_train[0], bins=50, alpha=0.5, density=True, label=tr,color = blue,hatch=r'\\')
    ax.hist(uncond_densities_test[0], bins=50, alpha=0.5, density=True, label=te,color = green,hatch=r'//')
    ax.hist(uncond_densities_ood[0], bins=50, alpha=0.5, density=True, label=ood,color = red,  hatch=r'||'   )
    total_distances = np.concatenate([uncond_densities_test[0], uncond_densities_ood[0]])
    results = compute_all_metrics(total_distances, labels, pred)
    method = f"LogP0_UNcond"
    _save_csv(results, tag, output_folder, 0,method)
    ax.set_title("$\log p_0(z)$; AUROC = " + str(results[1].round(3)))
    ax.set_xlabel("Log-Likelihood")   # ← X‐axis label
    ax.set_ylabel("UNcond. density")
    ax.legend()


    # ----------------- Plot 5: logpT of UNcond density-----------------
    ax = axs[1]
    #ax.hist(uncond_densities_gen[1], bins=100, alpha=0.5, density=True, label="gen")
    ax.hist(uncond_densities_train[1], bins=50, alpha=0.5, density=True, label=tr,color = blue,hatch=r'\\')
    ax.hist(uncond_densities_test[1], bins=50, alpha=0.5, density=True, label=te,color = green,hatch=r'//')
    ax.hist(uncond_densities_ood[1], bins=50, alpha=0.5, density=True, label=ood,color = red,hatch=r'||')
    total_distances = np.concatenate([uncond_densities_test[1], uncond_densities_ood[1]])
    results = compute_all_metrics(total_distances, labels, pred)
    method = f"LogPT_UNcond"
    _save_csv(results, tag, output_folder, 0,method)
    ax.set_xlabel("Log-Likelihood")   # ← X‐axis label
    ax.set_ylabel("UNcond. density")
    ax.set_title("$\log p_T(z)$; AUROC = " + str(results[1].round(3)))
    ax.legend()
    
    if ind_name == "cif10":
        title_name = "CIFAR-10"
    elif ind_name == "cif100":
        title_name = "CIFAR-100"
    else:
        title_name = "ImageNet200"
    #plt.tight_layout()
    #plt.suptitle(f" Densities of epoch = {epoch}, VDM trained = {title_name}, OOD = {tag}")
    plt.savefig(f"{output_folder}/LDM_ood_densities_{ind_name}_{tag}_{epoch}.png", dpi=300)
    plt.clf()
    plt.close()



In [61]:
# a function that draws a column of figures in the figure containing all the datasets and given methods
def draw_densities_big(axs, tag="", running_index=0, epoch=2999, acts_input=None,
                ind_name=None,
                output_folder=None,input_folder = None):
    ood_softs = np.load(f"{acts_input}/resnet18_{ind_name}_{tag}_softs.npy")
    #train_softs = np.load(f"{acts_input}/resnet18_{ind_name}_train_softs.npy")
    test_softs = np.load(f"{acts_input}/resnet18_{ind_name}_test_softs.npy")

    ood_y = np.load(f"{acts_input}/resnet18_{ind_name}_{tag}_y.npy")
    test_y = np.load(f"{acts_input}/resnet18_{ind_name}_test_y.npy")
    
    #densities_gen = np.load(f'{output_folder}/densities_gen.npy')
    #uncond_densities_gen = np.load(f'{output_folder}/uncond_densities_gen.npy')
    
    densities_test = np.load(f'{input_folder}/densities_test.npy')
    uncond_densities_test = np.load(f'{input_folder}/uncond_densities_test.npy')

    densities_train = np.load(f'{input_folder}/densities_train.npy')
    uncond_densities_train = np.load(f'{input_folder}/uncond_densities_train.npy')
    
    densities_ood = np.load(f'{input_folder}/densities_{tag}.npy')
    uncond_densities_ood = np.load(f'{input_folder}/uncond_densities_{tag}.npy')


    labels = np.concatenate([test_y, ood_y])
    pred = np.concatenate([test_softs.argmax(1), ood_softs.argmax(1)])
    
   # ----------------- Plot 3: bpd of cond density-----------------
    ax = axs[0,running_index]
    ax.hist(densities_train[0], bins=50, alpha=0.5, density=True,color = blue,  label=tr ,hatch=r'\\')
    ax.hist(densities_test[0], bins=50, alpha=0.5, density=True, color = green,label=te ,hatch=r'//')
    ax.hist(densities_ood[0], bins=50, alpha=0.5, density=True, color = red  ,label=ood,hatch=r'||')
    total_distances = np.concatenate([densities_test[0], densities_ood[0]])
    results = compute_all_metrics(total_distances, labels, pred)
    method = f"LogP0_cond"
    _save_csv(results, tag, output_folder, 0,method)
    ax.set_title(str(tag) + "\nAUROC = " + str(results[1].round(3)))
    if running_index == 0:
        ax.set_ylabel("Cond. density of $\log p_0(z)$")
        ax.legend()
    

    # ----------------- Plot 4: logp0 of UNcond density-----------------
    ax = axs[1,running_index]
    #ax.hist(uncond_densities_gen[0], bins=100, alpha=0.5, density=True, label="gen")
    ax.hist(uncond_densities_train[0], bins=50, alpha=0.5, density=True ,label=tr , color = blue, hatch=r'\\' )
    ax.hist(uncond_densities_test[0], bins=50, alpha=0.5, density=True  ,label=te , color = green, hatch=r'//' )
    ax.hist(uncond_densities_ood[0], bins=50, alpha=0.5, density=True   ,label=ood ,color = red  , hatch=r'||' )
    total_distances = np.concatenate([uncond_densities_test[0], uncond_densities_ood[0]])
    results = compute_all_metrics(total_distances, labels, pred)
    method = f"LogP0_UNcond"
    _save_csv(results, tag, output_folder, 0,method)
    ax.set_title("AUROC = " + str(results[1].round(3)))
    if running_index == 0:
        ax.set_ylabel("UNcond. density of $\log p_0(z)$")
        ax.legend()

    ax = axs[2,running_index]
    ax.hist(densities_train[1], bins=50, alpha=0.5, density=True,color = blue,  label=tr ,hatch=r'\\')
    ax.hist(densities_test[1], bins=50, alpha=0.5, density=True, color = green,label=te ,hatch=r'//')
    ax.hist(densities_ood[1], bins=50, alpha=0.5, density=True, color = red  ,label=ood,hatch=r'||')
    total_distances = np.concatenate([densities_test[1], densities_ood[1]])
    results = compute_all_metrics(total_distances, labels, pred)
    method = f"LogPT_cond"
    _save_csv(results, tag, output_folder, 0,method)
    ax.set_title("AUROC = " + str(results[1].round(3)))
    if running_index == 0:
        ax.set_ylabel("Cond. density of $\log p_T(z)$")
        ax.legend()



    # ----------------- Plot 5: logpT of UNcond density-----------------
    ax = axs[3,running_index ]
    #ax.hist(uncond_densities_gen[1], bins=100, alpha=0.5, density=True, label="gen")
    ax.hist(uncond_densities_train[1], bins=50, alpha=0.5, density=True, label=tr,color = blue,hatch=r'\\')
    ax.hist(uncond_densities_test[1], bins=50, alpha=0.5, density=True, label=te,color = green,hatch=r'//')
    ax.hist(uncond_densities_ood[1], bins=50, alpha=0.5, density=True, label=ood,color = red,hatch=r'||')
    total_distances = np.concatenate([uncond_densities_test[1], uncond_densities_ood[1]])
    results = compute_all_metrics(total_distances, labels, pred)
    method = f"LogPT_UNcond"
    _save_csv(results, tag, output_folder, 0,method)
    ax.set_title("AUROC = " + str(results[1].round(3)))
    if running_index == 0:
        ax.set_ylabel("UNcond. density of $\log p_T(z)$")
        ax.legend()
    ax.set_xlabel("Log-Likelihood")   # ← X‐axis label


    if ind_name == "cif10":
        title_name = "CIFAR-10"
    elif ind_name == "cif100":
        title_name = "CIFAR-100"
    else:
        title_name = "ImageNet200"

In [42]:
# a function to draw the diffusion loss over time
def draw_basics(tag="", ldm_version=10, epoch=2999, reps=25, num_step=20,
                acts_input=None,
                ind_name=None,
                output_folder=None, j = 0,input_folder=None):
    dif_test = np.load(f'{input_folder}/dif_loss_test_steps_{num_step}_reps_{reps}_{j}.npy')

    dif_train = np.load(f'{input_folder}/dif_loss_train_steps_{num_step}_reps_{reps}_{j}.npy')
    
    dif_ood = np.load(f'{input_folder}/dif_loss_{tag}_steps_{num_step}_reps_{reps}_{j}.npy')

    # Create a figure with 6 subplots arranged in 2 rows x 3 columns
    fig, ax = plt.subplots(1, 1, figsize=(7, 5))
    plt.rcParams.update({
            'font.size': 12,          # base font size
            'axes.titlesize': 14,     # title font size
            'axes.labelsize': 11,     # x– and y–label font size
            'xtick.labelsize': 12,    # tick label font size
            'ytick.labelsize': 12,    # tick label font size
            'legend.fontsize': 14,    # legend font size
            'figure.titlesize': 12    # figure title
        })

    # ----------------- Plot 1: Diff Loss Over Time -----------------
    x = np.linspace(1., 0., num_step)
    diff_array = [dif_train,dif_test,dif_ood]
    diff_color = [blue,green,red]
    diff_labels = [tr,te,ood]
    for i in range(3):
        mean_diff = diff_array[i].mean(1)
        ax.plot(x, mean_diff, alpha=1, color=diff_color[i], label=diff_labels[i])
        upper_b = mean_diff + diff_array[i].std(1)
        lower_b = mean_diff - diff_array[i].std(1)
        ax.plot(x, lower_b, alpha=0.3, color=diff_color[i])
        ax.plot(x, upper_b, alpha=0.3, color=diff_color[i])
        ax.fill_between(x, lower_b, upper_b, alpha=0.3, color=diff_color[i])
    if ind_name == "cif10":
        title_name = "CIFAR-10"
    elif ind_name == "cif100":
        title_name = "CIFAR-100"
    else:
        title_name = "ImageNet200"
    ax.set_title(f"VDM trained = {title_name}; epoch {epoch}, OOD = {tag}")
    ax.set_xlabel("Time t")   # ← X‐axis label
    ax.set_ylabel("Diffusion loss")
    ax.legend()

    #plt.tight_layout()
    
    #plt.suptitle(f"VDM trained = {title_name}; epoch {epoch}, OOD = {tag}")
    plt.savefig(f"{output_folder}/LDM_ood_rep_{reps}_{tag}.png", dpi=300,)
    plt.clf()
    plt.close()



In [76]:
# a function that draws a column of diffusion loss vs TKDL plot for each dataset and method and given timestep
def draw_basics_big(axs,tag="", running_index=0, epoch=2999, reps=25, num_step=10,
                acts_input=None,
                gen_softs=None,
                ind_name=None,
                output_folder=None,top_k=10,t_lim=0.,input_folder = None):
    ood_softs = np.load(f"{acts_input}/resnet18_{ind_name}_{tag}_softs.npy")
    test_softs = np.load(f"{acts_input}/resnet18_{ind_name}_test_softs.npy")

    ood_y = np.load(f"{acts_input}/resnet18_{ind_name}_{tag}_y.npy")
    test_y = np.load(f"{acts_input}/resnet18_{ind_name}_test_y.npy")
    # Load data
    labels = np.concatenate([test_y, ood_y])
    pred = np.concatenate([test_softs.argmax(1), ood_softs.argmax(1)]) 
    count = -1  
    for t_ind in range(num_step):
        t_ind_nr = 1. - t_ind*(1-t_lim)/num_step

        if t_ind_nr < 0.9:
            continue
        count+=1
        dif_test,logp_test,rec_test = get_time_t("test",top_k,num_step,reps,t_ind,input_folder)
        dif_train,logp_train,rec_train = get_time_t("train",top_k,num_step,reps,t_ind,input_folder)
        dif_ood,logp_ood,rec_ood = get_time_t(tag,top_k,num_step,reps,t_ind,input_folder) #shape (top_k,bs)
    # ----------------- Plot 1: Dif Loss  -----------------
        ax = axs[count,running_index]
        ax.hist(dif_train.min(0), bins=50, alpha=0.5, density=True, label=tr,color = blue,hatch=r'\\')
        ax.hist(dif_test.min(0), bins=50, alpha=0.5, density=True, label=te,color = green,hatch=r'//')
        ax.hist(dif_ood.min(0), bins=50, alpha=0.5, density=True, label=ood, color = red,hatch=r'||')
        total_distances = np.concatenate([dif_test.min(0), dif_ood.min(0)])
        results = compute_all_metrics(total_distances, labels, pred)
        auc =results[1].round(3)
        if results[1].round(3) < 0.5:
            auc = (1- auc).round(3)
        method = f"L_diff_{t_ind_nr}"
        _save_csv(results, tag, output_folder, t_ind_nr,method)
        if count == 0:
            ax.set_title(str(tag) + "\nAUROC = " + str(auc))
        else:
            ax.set_title("AUROC = " + str(auc))
        if running_index == 0:
            ax.set_ylabel(f"Dif. loss; t={t_ind_nr}")
            ax.legend()
        
        sm_test= softmax_smaller_better(dif_test.T).max(1) 
        sm_train = softmax_smaller_better(dif_train.T).max(1) 
        sm_ood = softmax_smaller_better(dif_ood.T).max(1) 
        count += 1
        ax = axs[count,running_index]
        ax.hist(sm_train, bins=50, alpha=0.5, density=True, label=tr,color = blue,hatch=r'\\')
        ax.hist(sm_test, bins=50, alpha=0.5, density=True, label=te,color = green,hatch=r'//')
        ax.hist(sm_ood, bins=50, alpha=0.5, density=True, label=ood,color = red,hatch=r'||')
        total_distances = np.concatenate([sm_test, sm_ood])
        results = compute_all_metrics(total_distances, labels, pred)
        auc =results[1].round(3)
        if results[1].round(3) < 0.5:
            auc = (1- auc).round(3)
        method = f"L_diffSM_{t_ind_nr}"
        _save_csv(results, tag, output_folder, t_ind_nr,method)
        ax.set_title("AUROC = " + str(auc))
        if running_index == 0:
            ax.set_ylabel(f"TKDL; t={t_ind_nr}")
            ax.legend()
        if count == 5:
            ax.set_xlabel("Score")   # ← X‐axis label

       


In [ ]:
import yaml

##for drawing the all methods and datasets plots

# Load configuration
with open('./LDM_tests/config_2.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Set parameters from config
ldm_version = config["ldm_version"]
special_index = config["special_index"]
ind_name = config["ind_name"]

output_folder = config["output_folder"].format(ind_name=ind_name,special_index=special_index)
input_folder = config["input_folder"].format(ind_name=ind_name,special_index=special_index)


if not os.path.exists(output_folder):
        # Create the folder (including any necessary parent directories)
        os.makedirs(output_folder)
        print(f"Folder created: {output_folder}")
else:
    print(f"Folder already exists: {output_folder}")

acts_input = config["acts_input"].format(ind_name=ind_name)
epoch = config["epoch"]
s_version = config["s_version"]
num_step_time = config["num_step_time"]
num_classes = config["num_classes"]
top_k = config["top_k"]
shall_draw_densities = config["shall_draw_densities"]

shall_draw_basics = config["shall_draw_basics"]
btb_on = config["btb_on"]
ind_tags = config["ind_tags"]
ood_tags = config["ood_tags"]
repetitions = config["repetitions"]
t_lim = config["t_lim"]
gen_softs = None

if shall_draw_densities:
    print("starting to draw densities....")
    plt.rcParams.update({
            'font.size': 12,          # base font size
            'axes.titlesize': 26,     # title font size
            'axes.labelsize': 25,     # x– and y–label font size
            'xtick.labelsize': 14,    # tick label font size
            'ytick.labelsize': 14,    # tick label font size
            'legend.fontsize': 16,    # legend font size
            'figure.titlesize': 10    # figure title
        })   
    # Create a figure with 6 subplots arranged in 2 rows x 3 columns
    fig, axs = plt.subplots(4, len(ood_tags), figsize=(6*len(ood_tags), 4*5),constrained_layout=True)
    for i,tag in enumerate(ood_tags):
        print(tag)
        draw_densities_big(axs,tag,i,epoch,acts_input,ind_name,output_folder,input_folder)
    plt.savefig(f"{output_folder}/LDM_ood_densities_{ind_name}_{epoch}_all.png", dpi=300)
    plt.clf()
    plt.close()
    print("...drawing done.")
    save_table(output_folder)
if shall_draw_basics:
    print("starting to draw basics....")
    plt.rcParams.update({
            'font.size': 12,          # base font size
            'axes.titlesize': 26,     # title font size
            'axes.labelsize': 28,     # x– and y–label font size
            'xtick.labelsize': 14,    # tick label font size
            'ytick.labelsize': 14,    # tick label font size
            'legend.fontsize': 16,    # legend font size
            'figure.titlesize': 10    # figure title
        })   
    # Create a figure with 6 subplots arranged in 2 rows x 3 columns
    fig, axs = plt.subplots(6, len(ood_tags), figsize=(len(ood_tags)*6, 6*5),constrained_layout=True)
    for i, tag in enumerate(ood_tags):
        print(tag)
        draw_basics_big(axs,tag, i,epoch,repetitions,num_step_time,acts_input,gen_softs,ind_name,output_folder,top_k=top_k,t_lim=t_lim,input_folder=input_folder)
    plt.savefig(f"{output_folder}/LDM_ood_basics_{ind_name}_{epoch}_all.png", dpi=300)
    plt.clf()
    plt.close()
    print("...drawing done.")
    save_table(output_folder)






In [ ]:
import yaml
#### for drawing the plots for each ood dataset separately
# Load configuration
with open('./LDM_tests/config_2.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Set parameters from config
ldm_version = config["ldm_version"]
special_index = config["special_index"]
ind_name = config["ind_name"]

output_folder = config["output_folder"].format(ind_name=ind_name,special_index=special_index)
input_folder = config["input_folder"].format(ind_name=ind_name,special_index=special_index)


if not os.path.exists(output_folder):
        # Create the folder (including any necessary parent directories)
        os.makedirs(output_folder)
        print(f"Folder created: {output_folder}")
else:
    print(f"Folder already exists: {output_folder}")

acts_input = config["acts_input"].format(ind_name=ind_name)
epoch = config["epoch"]
s_version = config["s_version"]
num_step_time = config["num_step_time"]
num_classes = config["num_classes"]
top_k = config["top_k"]
shall_draw_densities = config["shall_draw_densities"]

shall_draw_basics = config["shall_draw_basics"]
btb_on = config["btb_on"]
ind_tags = config["ind_tags"]
ood_tags = config["ood_tags"]
repetitions = config["repetitions"]
t_lim = config["t_lim"]

gen_softs = None

if shall_draw_densities:
    print("starting to draw densities....")
    for tag in ood_tags:
        print(tag)
        draw_densities(tag,ldm_version,epoch,acts_input,gen_softs,ind_name,output_folder,input_folder)
    print("...drawing done.")
    save_table(output_folder)
if shall_draw_basics:
    print("starting to draw basics....")
    for tag in ood_tags:
        print(tag)
        #draw_class_wise(tag,ldm_version,epoch,repetitions,num_step_time,acts_input,gen_softs,ind_name,output_folder)
        draw_basics(tag, ldm_version,epoch,repetitions,num_step_time,acts_input,ind_name,output_folder,j=0,input_folder=input_folder)
        if top_k !=1:
            draw_basics_cw(tag, ldm_version,epoch,repetitions,num_step_time,acts_input,gen_softs,ind_name,output_folder,top_k=top_k,t_lim=t_lim,input_folder=input_folder)
    print("...drawing done.")
    save_table(output_folder)

